In [1]:
# make sure ultralytics, cv2 are installed

# imports
import cv2
from ultralytics import YOLO
from collections import defaultdict
import numpy as np

In [2]:
# pretrained model
model = YOLO("yolov8n.pt")

In [3]:
# open video file
video_1 = "car_track.png"
cap = cv2.VideoCapture(video_1)
# Define the codec and create VideoWriter object
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))
# Store the track history
track_history = defaultdict(lambda: [])

In [4]:
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
            
        # Write the annotated frame to the output video
        out.write(annotated_frame)
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

/home/matteo/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



0: 384x640 1 train, 178.8ms
Speed: 15.4ms preprocess, 178.8ms inference, 22.2ms postprocess per image at shape (1, 3, 384, 640)


QObject::moveToThread: Current thread (0x15425c0) is not the object's thread (0xa9d91bb0).
Cannot move to target thread (0x15425c0)

QObject::moveToThread: Current thread (0x15425c0) is not the object's thread (0xa9d91bb0).
Cannot move to target thread (0x15425c0)

QObject::moveToThread: Current thread (0x15425c0) is not the object's thread (0xa9d91bb0).
Cannot move to target thread (0x15425c0)

QObject::moveToThread: Current thread (0x15425c0) is not the object's thread (0xa9d91bb0).
Cannot move to target thread (0x15425c0)

QObject::moveToThread: Current thread (0x15425c0) is not the object's thread (0xa9d91bb0).
Cannot move to target thread (0x15425c0)

QObject::moveToThread: Current thread (0x15425c0) is not the object's thread (0xa9d91bb0).
Cannot move to target thread (0x15425c0)

QObject::moveToThread: Current thread (0x15425c0) is not the object's thread (0xa9d91bb0).
Cannot move to target thread (0x15425c0)

QObject::moveToThread: Current thread (0x15425c0) is not the object's

In [42]:
import csv

with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Frame','Class ID', 'Object Name','Confidence', 'X1', 'Y1', 'X2', 'Y2'])

# Initialize a list to store detected objects
detected_objects = []
i = 0

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()
        
        ###########################
        
        ### JOES CODE TO WRITE TO FILE
        
        ################################
        
        detected_objects = results[0].boxes
        csv_file_name = 'detected_objects.csv'
        object_names = results[0].names
        with open(csv_file_name, mode='a', newline='') as file:
            writer = csv.writer(file)
            for box in detected_objects:
                class_id = box.cls[0].item()
                conf = box.conf[0].item()
                cords = box.xyxy[0].tolist()  # formatted as [x1, y1, x2, y2]
                object_name = object_names.get(class_id, 'Unknown')

                # Write the object data to the CSV file
                writer.writerow([i, class_id, object_name, conf, *cords])

                
        #####################
        
        # VISUALIZE
        
        ######################
        '''
        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
            
            # Add detected object information to the list
            detected_objects.append([i, track_id, x, y, w, h])

        # Write the annotated frame to the output video
        out.write(annotated_frame)
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        '''

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break
    i += 1

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 12 cars, 1 truck, 7.7ms
Speed: 1.8ms preprocess, 7.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 truck, 5.5ms
Speed: 1.7ms preprocess, 5.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 truck, 6.1ms
Speed: 1.6ms preprocess, 6.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 1 bus, 1 truck, 5.4ms
Speed: 2.5ms preprocess, 5.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 7.9ms
Speed: 2.3ms preprocess, 7.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 1 truck, 20.8ms
Speed: 4.2ms preprocess, 20.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 21.3ms
Speed: 5.2ms preprocess, 21.3ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 20.9ms
Speed: 4.8ms prepr

Speed: 1.5ms preprocess, 6.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 truck, 6.0ms
Speed: 1.4ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 truck, 6.7ms
Speed: 1.4ms preprocess, 6.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 bus, 5.0ms
Speed: 1.4ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 truck, 8.7ms
Speed: 1.9ms preprocess, 8.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 bus, 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 1 bus, 6.1ms
Speed: 1.9ms preprocess, 6.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 1 bus, 1 traffic light, 5.6ms
Speed: 2.0ms preprocess, 5.6ms inference, 1.1ms postprocess per ima


0: 384x640 9 cars, 1 bus, 1 truck, 6.2ms
Speed: 2.2ms preprocess, 6.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 6.2ms
Speed: 2.9ms preprocess, 6.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 5.2ms
Speed: 1.8ms preprocess, 5.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 5.2ms
Speed: 1.5ms preprocess, 5.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 5.6ms
Speed: 1.4ms preprocess, 5.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 3 trucks, 5.0ms
Speed: 1.4ms preprocess, 5.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 4 trucks, 5.0ms
Speed: 1.4ms preprocess, 5.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 4 trucks, 5.0ms
Speed: 

Speed: 1.7ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 bus, 1 truck, 6.6ms
Speed: 1.7ms preprocess, 6.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 1 bus, 1 truck, 6.5ms
Speed: 1.7ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 6.7ms
Speed: 1.8ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 bus, 2 trucks, 6.5ms
Speed: 1.7ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 6.5ms
Speed: 1.7ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 6.5ms
Speed: 1.7ms preprocess, 6.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 1 bus, 1 truck, 6.5ms
Speed: 1.7ms preprocess, 6.5ms in


0: 384x640 8 cars, 8.9ms
Speed: 2.3ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 9.0ms
Speed: 2.3ms preprocess, 9.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 9.0ms
Speed: 2.3ms preprocess, 9.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 9.0ms
Speed: 2.3ms preprocess, 9.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 8.9ms
Speed: 3.1ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 8.9ms
Speed: 2.3ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 9.0ms
Speed: 3.1ms preprocess, 9.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 8.9ms
Speed: 3.0ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 8.9

...

Speed: 1.4ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 5.4ms
Speed: 2.2ms preprocess, 5.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 5.3ms
Speed: 1.5ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 6.0ms
Speed: 1.8ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 5.2ms
Speed: 1.9ms preprocess, 5.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 1 truck, 5.6ms
Speed: 1.5ms preprocess, 5.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 5.4ms
Speed: 1.7ms preprocess, 5.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 2 trucks, 5.3ms
Speed: 1.6ms preprocess, 5.3ms inference, 1.0ms post

Speed: 1.6ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 5.4ms
Speed: 1.7ms preprocess, 5.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 5.2ms
Speed: 1.9ms preprocess, 5.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 5.4ms
Speed: 1.5ms preprocess, 5.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 5.5ms
Speed: 1.6ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 5.3ms
Speed: 2.2ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 5.8ms
Speed: 1.5ms preprocess, 5.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 5.4ms
Speed: 1.5ms preprocess, 5.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 

Speed: 1.7ms preprocess, 5.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 3 trucks, 5.3ms
Speed: 2.2ms preprocess, 5.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 3 trucks, 5.5ms
Speed: 2.3ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 3 trucks, 5.2ms
Speed: 1.9ms preprocess, 5.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 3 trucks, 5.1ms
Speed: 2.2ms preprocess, 5.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 3 trucks, 5.4ms
Speed: 2.6ms preprocess, 5.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 3 trucks, 5.1ms
Speed: 2.3ms preprocess, 5.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 3 trucks, 5.3ms
Speed: 1.9ms preprocess, 5.3ms inference, 1.0ms postprocess per image at sha


0: 384x640 10 cars, 2 buss, 1 truck, 6.0ms
Speed: 1.8ms preprocess, 6.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 5.6ms
Speed: 1.4ms preprocess, 5.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 5.2ms
Speed: 1.8ms preprocess, 5.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 1 bus, 2 trucks, 5.8ms
Speed: 1.5ms preprocess, 5.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 2 trucks, 5.8ms
Speed: 1.5ms preprocess, 5.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 2 trucks, 5.5ms
Speed: 1.6ms preprocess, 5.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 bus, 2 trucks, 5.6ms
Speed: 1.4ms preprocess, 5.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 1 


0: 384x640 15 cars, 27.6ms
Speed: 6.5ms preprocess, 27.6ms inference, 4.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 21.4ms
Speed: 5.4ms preprocess, 21.4ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 20.2ms
Speed: 4.5ms preprocess, 20.2ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 17.5ms
Speed: 4.2ms preprocess, 17.5ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 19.2ms
Speed: 4.4ms preprocess, 19.2ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 cars, 19.1ms
Speed: 5.1ms preprocess, 19.1ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 20.4ms
Speed: 4.4ms preprocess, 20.4ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 18.1ms
Speed: 4.4ms preprocess, 18.1ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)
